Trong notebook này, ta sẽ:
 - Làm quen với thư viện sklearn dùng cho học máy
 - Làm quen với một số kỹ thuật phổ biến cho học máy có giám sát

Học có giám sát (supervised learning) chia làm hai loại chính: Regression (hồi quy), và Classification (phân loại).

In [ ]:
# Ta load những thư viện cần sử dụng

import os
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

# Dưới đây là các phép đo lỗi phổ biến cho bài toán regression
# các phép đo error này nếu giá trị càng nhỏ tức là mô hình càng fit tốt với dữ liệu
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Các phép đo phổ biến cho bài toán classification
from sklearn.metrics import accuracy_score, f1_score

import warnings
warnings.filterwarnings('ignore')

# Phần 1: Bài toán Regression

In [ ]:
os.listdir("data")

In [ ]:
# Đọc dữ liệu giá bất động sản montreal
# Mục tiêu: Dự đoán giá bán (ask price) của các ngôi nhà dựa trên 39 tham số
# Ta dùng thư viện pandas để đọc bảng csv với hàm read_csv

df_montreal = pd.read_csv("data/montreal_housing.csv")

In [ ]:
# Kiểm tra kích thước dữ liệu bằng câu lệnh sau
df_montreal.shape

Kết quả cho thấy dataframe có 9717 hàng và 40 cột

In [ ]:
# Sựu dụng phương thức .head() để view bảng dữ liệu, hay còn gọi là dataframe
df_montreal.head()

In [ ]:
# Tách dataframe thành hai phần x và y (đầu vào và đầu ra của mô hình học máy) 

x = df_montreal.iloc[:, :-1]
y = df_montreal["askprice"]

Trong một project học máy, dữ liệu bao giờ cũng sẽ được chia làm ít nhất hai phần:
 - Phần thứ nhất dùng để huấn luyện mô hình (training data)
 - Phần thứ hai dùng để kiểm tra hiệu năng của mô hình với các hàm đo performance (testing data)

So sánh kết quả performance trên training và testing data sẽ cho ta biết:
 - Mô hình có đang fit tốt hay không
 - Có bị mắc các lỗi như overfit, underfit hay không.

Từ đó, ta sẽ có phương án huấn luyện tốt hơn

In [ ]:
# Tách dữ liệu thành training và testing set bằng hàm train_test_split của thư viện sklearn
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.25, random_state = 42)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:

# Để huấn luyện mô hình Linear Regression, ta dùng class Linear Regression đã import ở phía trên
linear_reg = LinearRegression(normalize= True)

# Sử dụng phương thức ".fit" để training
linear_reg.fit(x_train, y_train)

Sử dụng độ đo mean_squared_error để kiểm tra hiệu năng mô hình

In [ ]:
# Testing models
y_pred_train = linear_reg.predict(x_train)
y_pred = linear_reg.predict(x_test)

print(f"MSE on training: {mean_squared_error(y_train,y_pred_train):.3f}, on testing: {mean_squared_error(y_test, y_pred):.3f}")
print(f"MAE on training: {mean_absolute_error(y_train, y_pred_train):.3f}, on testing: {mean_absolute_error(y_test, y_pred):.3f}")

Sử dụng Linear Regression không tốt cho trường hợp này lắm => Thử những mô hình cao cấp hơn

Ta sẽ sử dụng hai mô hình nâng cao của Linear Regression là Ridge và Lasso, hai mô hình này có thêm tham số chuẩn hóa giúp mô hình bớt bị overfit hơn

Ban chưa cần quan tâm đến lý thuyết của hai mô hình này vội, vì chúng sẽ được dạy ở buổi 4

In [ ]:
from sklearn.linear_model import Ridge, Lasso

ridge = Ridge(alpha = 0.1, normalize= True)
ridge.fit(x_train, y_train)

In [ ]:
# Testing models
y_pred_train = ridge.predict(x_train)
y_pred = ridge.predict(x_test)

print(f"MSE on training: {mean_squared_error(y_train, y_pred_train):.3f}, on testing: {mean_squared_error(y_test, y_pred):.3f}")
print(f"MAE on training: {mean_absolute_error(y_train, y_pred_train):.3f}, on testing: {mean_absolute_error(y_test, y_pred):.3f}")

Ta thấy model đã giảm lỗi MSE, tuy nhiên vẫn khá cao.

In [ ]:
lasso = Lasso(alpha = 0.001, normalize= True)
lasso.fit(x_train, y_train)

In [ ]:
# Testing models
y_pred_train = lasso.predict(x_train)
y_pred = lasso.predict(x_test)

print(f"MSE on training: {mean_squared_error(y_train, y_pred_train):.3f}, on testing: {mean_squared_error(y_test, y_pred):.3f}")
print(f"MAE on training: {mean_absolute_error(y_train, y_pred_train):.3f}, on testing: {mean_absolute_error(y_test, y_pred):.3f}")

Ta có thể thấy với hồi quy Lasso và Ridge, model đã bớt bị overfit hơn rất nhiều. Tuy nhiên, độ chính xác của hai mô hình này phụ thuộc vào giá trị của tham số alpha

# Phần 2: Bài toán Classification

In [ ]:
df_iris = pd.read_csv("data/iris.csv", index_col = "Unnamed: 0")

In [ ]:
df_iris

In [ ]:
df_iris.shape

## Mục tiêu: Dự đoán loài hoa dựa vào 4 thông số từ sepal length đến petal width

### Nhận xét: dữ liệu có kích thước khá nhỏ nên tiến hành evaluation bằng cross validation

In [ ]:
from IPython.display import Image
Image('cross-validation.png')

Giải thích cross_validation:
    Ta chia dữ liệu ra làm n phần và tiến hành training n lần
    - Với mỗi một lần training ta lấy một phần trong n dataset nhỏ làm testing data, và (n-1) còn lại làm training data
    - Lần lượt huấn luyện cho đến khi cả n dataset nhỏ đều được dùng để làm test
    - Kết quả đo performance cuối cùng sẽ là trung bình của n lần đo performance trên các dataset nhỏ

In [ ]:
x = df_iris.iloc[:, :-1]
y = df_iris["Species"]

In [ ]:
y

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_encode = LabelEncoder()
y_new = label_encode.fit_transform(y)

In [ ]:
y_new

In [ ]:
label_encode.inverse_transform([0, 1, 2])

In [ ]:
from sklearn.model_selection import cross_validate

# scoring = {'precision': make_scorer(accuracy_score),
#             'f1': make_scorer(f1_score)}
scoring = ['precision_macro', 'recall_macro']

clf = LogisticRegression()

# Khi ta chạy hàm cross_calidate, kết quả trả về sẽ là một dictionary lưu trữ thông tin của quá trình huấn luyện
scores = cross_validate(clf, x, y_new, scoring=scoring, cv=5, return_train_score=True, return_estimator = True)

In [ ]:
sorted(scores.keys())

Trong đó:
- estimator là tham số của các mô hình đã được huấn luyện
- fit_time, score_time là thời gian huấn luyện và đánh giá cho mỗi lần validation
- test/train_<tên độ đo> là kết quả đo của độ đo performance trên tập train và test cho mỗi lần validation

In [ ]:
print(scores['train_precision_macro'])
print(scores['test_precision_macro'])

In [ ]:
# Mỗi giá trị test/train_precision_macro là một mảng numpy, ta có thể dùng phương thức ndarray.mean() để tìm độ
# chính xác trung bình
print(scores['train_precision_macro'].mean())
print(scores['test_precision_macro'].mean())

In [ ]:
# Kiểm tra độ đo recall trên train và test

print(scores['train_recall_macro'])
print(scores['test_recall_macro'])

In [ ]:
print(scores['train_recall_macro'].mean())
print(scores['test_recall_macro'].mean())

In [ ]:
# Gõ key estimator để  truy xuất ra các mô hình trong quá trình cross val
# Mỗi mô hình sau khi được huấn luyện xong sẽ được ký hiệu là một object như sau
scores["estimator"][0]

In [ ]:
# Gõ coeff_ để truy xuất ra trọng số của mỗi LogisticRegression object như sau
scores["estimator"][0].coef_

**Key takeaway:**

- Training và testing là hai phần không thể tách rời trong một project machine learning
- Ta có thể sử dụng thư viện sklearn để áp dụng các mô hình học máy và các kỹ thuật xử lý số liệu cho machine learning
- Các model học máy được lưu trữ dưới dạng các class trong thư viện sklearn
- Để tiến hành huấn luyện mô hình, ta khởi tạo một biến với class là tên mô hình đó (ví dụ LinearRegression, Logistic Regression) và sử dụng phương thức ".fit"
- Mỗi một loại bài toán sẽ có những độ đo phù hợp, ta có thể tìm implementation của các độ đo đó tại sub-package sklearn.metrics

Để hiểu thêm về cách sử dụng thư viện sklearn, ta có thể xem trang web của sklearn [tại đậy](https://scikit-learn.org/stable/index.html) 

Sách tham khảo về học máy với sklearn: [scikit-learn Cookbook](https://github.com/whoafridi/Machine-Learning-Books/blob/master/book/scikit-learn%20Cookbook%20-%20Second%20Edition.pdf)